# 实现RNN

In [1]:
#encoding:utf-8
import json
import time
import random
import numpy as np
import pandas as pd
import math
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import sys
from sklearn.externals import joblib
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import time
from torch.autograd import Variable
from glob import glob
import os
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import shutil
from torchvision import transforms
from torchvision import models
from torchtext import data, datasets
from nltk import ngrams
from torchtext.vocab import GloVe, Vectors
from collections import defaultdict
data_path = r'D:\kaggle\data\spooky-author-identification\a'[: -1]
data_path_inv = r'D:\kaggle\data\spooky-author-identification\a'[: -1]
data_path_word_vector = r'D:\kaggle\data\word_vector\a'[: -1]

import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
stop_words = stopwords.words('english')
from torch.nn import utils as nn_utils

In [2]:
def read_data():
    df_train = pd.read_csv(data_path_inv + r'train/train.csv')
    df_test = pd.read_csv(data_path + 'test/test.csv')
    df_sub = pd.read_csv(data_path + 'sample_submission/sample_submission.csv')
    return df_train, df_test, df_sub
df_train, df_test, df_sub = read_data()
print (df_train.shape, df_test.shape, df_sub.shape)
print (df_train.head(3), '\n\n', df_test.head(3), '\n\n', df_sub.head(3), )

(19579, 3) (8392, 2) (8392, 4)
        id                                               text author
0  id26305  This process, however, afforded me no means of...    EAP
1  id17569  It never once occurred to me that the fumbling...    HPL
2  id11008  In his left hand was a gold snuff box, from wh...    EAP 

         id                                               text
0  id02310  Still, as I urged our leaving Ireland with suc...
1  id24541  If a fire wanted fanning, it could readily be ...
2  id00134  And when they had broken down the frail door t... 

         id       EAP       HPL       MWS
0  id02310  0.403494  0.287808  0.308698
1  id24541  0.403494  0.287808  0.308698
2  id00134  0.403494  0.287808  0.308698


In [3]:
# 处理数据
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(df_train.author.values)
xtrain, xvalid, ytrain, yvalid = train_test_split(df_train.text.values, y, 
                                                  stratify = y, 
                                                  random_state = 2020, 
                                                  test_size = 0.1, shuffle = True)
xtest = df_test.text.values
print (xtrain.shape)
print (xvalid.shape)
print (xtest.shape)

(17621,)
(1958,)
(8392,)


In [4]:
# load the GloVe vectors in a dictionary:
embeddings_index = {}
f = open(data_path_word_vector + 'glove.840B.300d.txt', 'rb')
index = 0
pre_time = time.time()
for line in f: # tqdm(f):
    index += 1
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    if index % 500000 == 0:
        print ('index: {:}, time:{:}'.format(index, time.time() - pre_time))
        pre_time = time.time()
f.close()
print('Found %s word vectors.' % len(embeddings_index))

index: 500000, time:41.491108894348145
index: 1000000, time:40.718759059906006
index: 1500000, time:42.45925521850586
index: 2000000, time:42.93292021751404
Found 2196016 word vectors.


In [5]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            torch_tmp = embeddings_index[str.encode(w)]
            M.append(torch_tmp)
        except:
            continue
    return M

xtrain_glove = [sent2vec(x) for x in xtrain]
len_max = 0
for x in xtrain_glove:
    len_max = max(len_max, len(x))
print (len_max)
for i in range(len(xtrain_glove)):
    for j in range(len_max - len(xtrain_glove[i])):
        xtrain_glove[i].append(np.zeros(300))
len_min = len_max
for x in xtrain_glove:
    len_min = min(len_min, len(x)) 
print (len_min)

xtrain_glove = np.array(xtrain_glove)
print (xtrain_glove.shape)
xvalid_glove = np.array([sent2vec(x) for x in xvalid])
print (xvalid_glove.shape)
xtest_glove = np.array([sent2vec(x) for x in xtest])
print (xtest_glove.shape)

# # scale the data before any neural net:
# scl = preprocessing.StandardScaler()
# xtrain_glove_scl = scl.fit_transform(xtrain_glove)
# xvalid_glove_scl = scl.transform(xvalid_glove)

print (xtrain_glove.shape, xvalid_glove.shape)
print (ytrain.shape, yvalid.shape)
print (ytrain[: 3], yvalid[: 3])
word_vector_size = len(embeddings_index[list(embeddings_index.keys())[0]])
label_size = 3

252
252
(17621, 252, 300)
(1958,)
(8392,)
(17621, 252, 300) (1958,)
(17621,) (1958,)
[1 0 1] [1 1 1]


In [6]:
num_inputs, num_hiddens, num_outputs = word_vector_size, 512, 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
    Y = torch.matmul(H, W_hq) + b_q
    # Y = Y.view(-1)
    return Y

def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

def data_iter_fn(xtrain, ytrain, batch_size, device):
    num_steps = len(xtrain_glove_scl) // batch_size
    x_out, y_out = [], []
    for epoch in range(num_steps):
        xval = torch.from_numpy(xtrain[i * batch_size: (i + 1) * batch_size, :]).cuda().type(torch.float32)
        yval = ytrain[i * batch_size: (i + 1) * batch_size]
        x_out.append(xval)
        y_out.append(yval)
    y_out = torch.from_numpy(np.array(y_out)).cuda().type(torch.float32)
    return x_out, y_out

def sgd(params, lr, batch_size):
    # 为了和原书保持一致，这里除以了batch_size，但是应该是不用除的，因为一般用PyTorch计算loss时就默认已经
    # 沿batch维求了平均了。
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.data

will use cuda


In [28]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            torch_tmp = list(embeddings_index[str.encode(w)])
            M.append(torch_tmp)
        except:
            continue
    return M

xtrain_glove = [sent2vec(x) for x in xtrain]
xvalid_glove = [sent2vec(x) for x in xvalid]
xtest_glove = [sent2vec(x) for x in xtest]

xtrain_lengths = torch.from_numpy(np.array([len(x) for x in xtrain_glove])).cuda()
print (xtrain_lengths.max())
max_length = 100
xtrain_tensor = torch.zeros((len(xtrain_glove), max_length, 300)).long().cuda()
for idx, (seq, seqlen) in enumerate(zip(xtrain_glove, xtrain_lengths)):
    seqlen = min(seqlen, max_length)
    xtrain_tensor[idx, :seqlen] = torch.LongTensor(seq[: seqlen, :])

print (typ(xtrain_tersor), xtrain_tersor.size())
word_vector_size = len(embeddings_index[list(embeddings_index.keys())[0]])
label_size = 3

tensor(252, device='cuda:0', dtype=torch.int32)


RuntimeError: $ Torch: not enough memory: you tried to allocate 3GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

In [ ]:
print (type(xtrain_glove), xtrain_glove.shape)
len_min = 10000000
len_max = 0
for i in range(len(xtrain_glove)):
    len_min = min(len_min, len(xtrain_glove[i]))
    len_max = max(len_max, len(xtrain_glove[i]))
    # print (type(xtrain_glove[i]), len(xtrain_glove[i]))
print (len_min, len_max)
print (len(xtrain_glove[0][0]))
print (len(xtrain_glove[1][-1]))
print (type(xtrain_glove[0]))
print (type(xtrain_glove[0][0]))

xtrain_glove = np.array(xtrain_glove)
print (xtrain_glove.shape)
print (type(xtrain_glove[0, :]))
print (type(xtrain_glove[0][0]))

In [ ]:
print (ytrain[: 20])
print (type(xtrain_glove), xtrain_glove.shape)
xtrain_tmp = [[xtrain_glove[i], ytrain[i]] for i in range(len(xtrain_glove))]
xtrain_tmp = np.array(sorted(xtrain_tmp, key = lambda x: len(x[0]), reverse = True))
xtrain_sort_len = xtrain_tmp[: , 0]
ytrain = xtrain_tmp[: , 1]
print (type(ytrain))
print (ytrain[: 20])
for i in range(10):
    print (i, len(xtrain_sort_len[i]))

In [ ]:
print (type(xtrain_glove), xtrain_glove.shape)
xtrain_torch = torch.from_numpy(xtrain_glove)
print (xtrain_torch.size())

In [ ]:
embed_input_x_packed = pack_padded_sequence(xtrain_glove, xtrain_len, batch_first=True)
encoder_outputs_packed, (h_last, c_last) = self.lstm(embed_input_x_packed)
encoder_outputs, _ = pad_packed_sequence(encoder_outputs_packed, batch_first=True)

# demo

In [ ]:
### 定义模型
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space，相当于一个全连接层
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))  # 把三维张量转化为二级张量
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

### 模型训练及预测
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()    # 调用时形式为：预测值(N*C),label(N)。其中N为序列中word数，C为label的类别数
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)  # 此处的inputs只能是一个sequence
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)    #一个sequence对应的词性标注list

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print("[result]tag_scores={}".format(tag_scores))